In [120]:
import pickle
import pandas as pd
from word2number import w2n
import re


In [119]:
# This cell only is generated by ChatGPT and modified by me

ocr_corrections = {'O': '0', 'I': '1', 'Z': '2', 'S': '5', 'B': '8', 'l': '1'}

def correct_ocr_errors(s):
    for char, replacement in ocr_corrections.items():
        s = s.replace(char, replacement)
    return s

def is_number(string):
    string = string.strip()
    
    pattern = re.compile(r'^-?\d+(\.\d+)?$')
    return bool(pattern.match(string))

def tonum(input_str):
    if isinstance(input_str, int):
        return input_str

    input_str = input_str.strip() 
    
    if is_number(input_str):
        try:
            return int(input_str)
        except ValueError:
            return float(input_str)
    
    corrected_str = correct_ocr_errors(input_str)
    
    if is_number(corrected_str):
        try:
            return int(corrected_str)
        except ValueError:
            return float(corrected_str)
    
    try:
        return w2n.word_to_num(input_str)
    except ValueError:
        return f"Error: '{input_str}' is not a valid number"


In [9]:
with open('data/expired_invoices.txt', 'r') as f:
    expired_invoices = [int(i) for i in f.read().strip().split(', ')]


In [141]:
class DataExtractor:
    type_str = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'}

    def __init__(self, pickle_file, expired):
        self.file = pickle_file
        self.expired = expired

    def load(self):
        with open(self.file, 'rb') as pkl:
            self.data = pickle.load(pkl)

    def transform(self):
        flat = []
        
        for inv in self.data:

            # no specs were given about cases with missing items list so just continue
            if 'items' not in inv.keys():
                continue

            inv_total = 0
            for i in inv['items']:
                inv_total += tonum(i['item']['unit_price']) * tonum(i['quantity'])

            for item in inv['items']:
                unit_price = tonum(item['item']['unit_price'])
                total_price = unit_price * tonum(item['quantity'])

                # no specs were given about cases with missing dates so just set them to 0
                try:
                    created_on = pd.to_datetime(pd.Series(inv['created_on'])).values[0]
                except ValueError:
                    created_on = pd.to_datetime(pd.Series('1970-01-01')).values[0]

                flat.append({'invoice_id': tonum(inv['id']), 'created_on': created_on, 'invoiceitem_id': tonum(item['item']['id']), 
                             'invoiceitem_name': item['item']['name'], 'type': self.type_str[tonum(item['item']['type'])], 'unit_price': unit_price, 
                             'total_price': total_price, 'percentage_in_invoice': total_price / inv_total, 'is_expired': inv['id'] in self.expired})
        
        return pd.DataFrame(flat)


In [142]:
initial = DataExtractor('data/invoices_new.pkl', expired_invoices)
initial.load()
flat = initial.transform()

In [143]:
# sorting
flat_sorted = flat.sort_values(by=['invoice_id', 'invoiceitem_id'], ascending=[True, True])
# saving to csv
flat_sorted.to_csv('flat_data.csv', index=False)
